In [1]:
import torch
from torch import nn
from transformers import GPT2Model, GPT2Config

# Load the predetermined embeddings matrix (e.g., from a numpy file)
pretrained_embeddings = np.load('path_to_pretrained_embeddings.npy')  # Shape should be (num_features, embedding_dim)

# Convert to PyTorch tensor
pretrained_embeddings = torch.tensor(pretrained_embeddings, dtype=torch.float32)


c:\Users\tniu\AppData\Local\miniconda3\envs\llm-base\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'np' is not defined

In [ ]:
class GPT2WithPretrainedPhoneticFeatures(nn.Module):
    def __init__(self, gpt2_model, pretrained_embeddings):
        super(GPT2WithPretrainedPhoneticFeatures, self).__init__()
        self.gpt2_model = gpt2_model
        
        # Create an embedding layer with pretrained weights
        num_features, embedding_dim = pretrained_embeddings.shape
        self.phonetic_embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        
        # Verify the embedding dimension matches the model's input size
        assert embedding_dim == config.n_embd, "Embedding dimension must match the model's embedding dimension."
        
        # Classifier to classify each token into one of three classes
        self.classifier = nn.Linear(config.n_embd, 3)  # 3 classes
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, phonetic_inputs):
        # Apply the embedding layer to convert phonetic features to embeddings
        embedded_inputs = self.phonetic_embedding(phonetic_inputs)
        # Pass the embedded inputs to the GPT-2 model
        gpt_outputs = self.gpt2_model(inputs_embeds=embedded_inputs)
        # Apply the classifier to each output token
        logits = self.classifier(gpt_outputs.last_hidden_state)
        # Apply softmax to get probabilities
        probabilities = self.softmax(logits)
        return probabilities

# Load the pre-trained GPT model
gpt2_model = GPT2Model(config)
# Initialize the modified GPT-2 model with pretrained embeddings
modified_gpt2 = GPT2WithPretrainedPhoneticFeatures(gpt2_model, pretrained_embeddings)


In [ ]:
import numpy as np

# Example data: phonetic feature indices (batch_size, seq_length) and labels (batch_size, seq_length)
phonetic_features_indices = np.random.randint(0, num_features, (32, 10))  # Example shape (32 samples, 10 words each)
labels = np.random.randint(0, 3, (32, 10))  # Example shape (32 samples, 10 words each), labels in {0, 1, 2}

# Convert to PyTorch tensors
phonetic_features_indices = torch.tensor(phonetic_features_indices, dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.long)


In [ ]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

# Define the optimizer and loss function
optimizer = Adam(modified_gpt2.parameters(), lr=5e-5)
criterion = CrossEntropyLoss()

# Training loop
epochs = 10  # Number of epochs

for epoch in range(epochs):
    modified_gpt2.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = modified_gpt2(phonetic_features_indices)

    # Compute the loss
    # Note: outputs is (batch_size, seq_length, num_classes), labels is (batch_size, seq_length)
    loss = criterion(outputs.view(-1, 3), labels.view(-1))

    # Backward pass
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Save the trained model
torch.save(modified_gpt2.state_dict(), 'modified_gpt2_with_pretrained_phonetic_embeddings.pt')
